In [2]:
import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

import sys

import fastparquet
import snappy

In [3]:
alldays = pd.read_csv('VehtoSep9.csv')

In [4]:
alldays

,Begin Date,Begin Time,End Time,Vehicle Location,Vehicle Type,Vehicle Characteristics,Transaction Characteristics,Bikeway Users Displaced,LZ Space Avail,Occupied while idle?,CNS?,TNC?,Notes,Duration
0,8/28/2019,12:00:00 PM,12:02:43 PM,NB right turn pocket,Delivery Van (< 20 feet),NaN,NaN,NaN,2.0,False,False,False,NaN,0:02:43
1,8/28/2019,12:00:00 PM,12:11:35 PM,NB right turn pocket,Delivery Van (< 20 feet),NaN,NaN,NaN,2.0,False,False,False,NaN,0:11:35
2,8/28/2019,12:04:21 PM,12:16:20 PM,Space 1,Car,Other significant characteristic (see note),Interaction with parking meter,NaN,NaN,True,False,False,"Two people, both get out",0:11:59
3,8/28/2019,12:29:16 PM,12:31:40 PM,Space 2,Pickup or SUV,Other significant characteristic (see note),No interaction with parking meter,NaN,1.0,False,True,False,"One person, comes back with a coffee",0:02:24
4,8/28/2019,12:32:19 PM,12:33:43 PM,NB right turn pocket,Car,Blinkers on,NaN,NaN,2.0,False,False,False,One person gets out,0:01:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
4990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
4991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
4992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN


In [3]:
# alldays.dropna(subset=['Begin Date'], inplace=True)
# alldays['Begin Date'] = alldays['Begin Date'].apply(zeropad_dates)
# alldays[alldays['Begin Date'] == '09/01/2019'].to_csv('partialSep1.csv')

In [4]:
def showall(df):
    #shows entire dataframe
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

In [5]:
def zeropad_dates(str_date):
    ##TODO speedup candidate?
    #pads month and day with zeroes
    padded_date = ''
    split_date = str_date.split('/')
    for value in split_date:
        if len(value) < 2:
            value = '0' + value
        padded_date += (value + '/')
    padded_date = padded_date[:-1]
    return padded_date

In [6]:
def datetime_from_str(string):
    #converts string format dates/times from spreadsheet into Python Datetime objects
    
    try:
        #print('ran, string:{}'.format(string))
        format_str = '%m/%d/%Y%I:%M:%S %p'
        to_dt = dt.datetime.strptime(string, format_str)
    except ValueError:
        #handles small portion of values in 24hr format
        try:
            #print('excepted, string:{}'.format(string))
            format_str = '%m/%d/%Y%H:%M:%S'
            to_dt = dt.datetime.strptime(string, format_str)
        
        except ValueError:
            #use dict to handle both text-described times
            text_times_dict = {'before 12':'12:00:00', 'end of the day':'22:30:00'}
            #print('excepted2, string:{}'.format(string))
            texttime = string[10:]
            time = text_times_dict[texttime]
            combined = string[:10] + time
            return datetime_from_str(combined)
    return to_dt

In [1]:
def classify_violator(row, st_time='18:00:00', end_time='22:00:00', legal_minutes=15):
    ##TODO speedup candidate?
    #classify a row as a violator or not
    #considers +15min parked from 18:00-22:30 a violation
    
    begin, end = row['Begin Date']+ row['Begin Time'], row['Begin Date'] + row['End Time']
    time_parked = {'Start':datetime_from_str(begin), 'End':datetime_from_str(end)}
    datestr = begin[:10]
    
    enf_start = datestr + st_time
    enf_end = datestr + end_time
    enforcement_times = {'Start':datetime_from_str(enf_start), 'End':datetime_from_str(enf_end)}
    
    latest_start = max(time_parked['Start'], enforcement_times['Start'])
    earliest_end = min(time_parked['End'], enforcement_times['End'])
    delta = (earliest_end - latest_start).seconds
    
    #maximum parking duration during enforcement interval
    legal_duration_seconds = legal_minutes*60
    max_observation = 60**2 * 11
    if (delta > legal_duration_seconds 
        and delta < max_observation 
        #Space 3 not part of loading zone
        and row['Vehicle Location'] in ['Space 1', 'Space 2']):
        
        return 'Violator'
    elif row['Vehicle Location'] == 'SB bike lane':
        return 'Bike Lane Blocking'
    else:
        return 'Likely Non-Violator' 
    
def classify_violators(df):
    #add boolean violator column to original (pre-timestamp) df. Also zeropads dates.
    
    df.dropna(subset=['Begin Time', 'End Time'], inplace=True)
    df['Begin Date'] = df['Begin Date'].apply(zeropad_dates)
    df['Violator'] = df.apply(classify_violator, axis=1)
    
    return df


In [8]:
def timestamps_from_interval(dt_start, dt_end):
    ##TODO speedup, may be tough to speedup datetime comparison... 
    #generates list of datetime objects for every second in interval
    timestamps = [dt_start]
    timestamp = dt_start
    while timestamp < dt_end:
        timestamp += dt.timedelta(seconds=1)
        timestamps = timestamps + [timestamp]
    return timestamps

def timestamps_from_row(row):
    #extracts interval from a row of the original spreadsheet
    start = row[1]['Begin Date'] + row[1]['Begin Time'] 
    end = row[1]['Begin Date'] + row[1]['End Time'] 
    
    dt_start = datetime_from_str(start)
    dt_end = datetime_from_str(end)

    return timestamps_from_interval(dt_start, dt_end)

def multiindex_for_row(row):
    #creates multiindex of timestamps and vehicle locations
    timestamps = timestamps_from_row(row)
    location = [row[1]['Vehicle Location']]
    locations = location * len(timestamps)
    tuples = list(zip(timestamps, locations))
    
    return pd.MultiIndex.from_tuples(tuples, names=['Timestamp', 'Vehicle_Location'])
    
def df_for_row(row):
    #generates multi-indexed (timestamped) dataframe from a single row of the original spreadsheet
    multiind = multiindex_for_row(row)
    row_df = pd.DataFrame(row[1]).swapaxes('index', 'columns')
    row_df = pd.concat([row_df]*len(multiind))
    row_df.index = multiind
    return row_df

In [9]:
def timestamp_df(df):
    ##TODO speedup, probably tough to get rid of the for loop here
    #Generates timestamped dataframe from original spreadsheet, after classify violators. Currently slow to run. 
    timestamped_df = pd.DataFrame()
    i = 0
    for row in df.iterrows():
        if i % 100 == 0:
            #TODO rewrite to include %complete, ETA?
            print('Processing Row: {}, {}% complete'.format(i, int((i/df.shape[0])*100)))
        i += 1
        timestamped_df = timestamped_df.append(df_for_row(row))
    return timestamped_df

In [10]:
def timestamp_and_classify_violators(df):
    return timestamp_df(classify_violators(df)) 

In [11]:
alldays_timestamped = timestamp_and_classify_violators(alldays)

Processing Row: 0, 0% complete
Processing Row: 100, 4% complete
Processing Row: 200, 9% complete
Processing Row: 300, 14% complete
Processing Row: 400, 19% complete
Processing Row: 500, 24% complete
Processing Row: 600, 29% complete
Processing Row: 700, 33% complete
Processing Row: 800, 38% complete
Processing Row: 900, 43% complete
Processing Row: 1000, 48% complete
Processing Row: 1100, 53% complete
Processing Row: 1200, 58% complete
Processing Row: 1300, 62% complete
Processing Row: 1400, 67% complete
Processing Row: 1500, 72% complete
Processing Row: 1600, 77% complete
Processing Row: 1700, 82% complete
Processing Row: 1800, 87% complete
Processing Row: 1900, 91% complete
Processing Row: 2000, 96% complete


In [12]:
alldays_timestamped.shape

(1486019, 15)

In [15]:
alldays_timestamped.xs('2019-09-01 18:31:48', level='Timestamp', drop_level=False)

Begin Date  Begin Time    End Time  \
Timestamp           Vehicle_Location                                           
2019-09-01 18:31:48 Space 3               09/01/2019  5:00:00 PM  6:32:42 PM   
                    Space 1               09/01/2019  6:16:07 PM  9:59:08 PM   
                    Space 2               09/01/2019  6:23:41 PM  7:43:29 PM   
                    NB right turn pocket  09/01/2019  6:28:52 PM  6:31:53 PM   
                    SB bike lane          09/01/2019  6:31:10 PM  6:33:20 PM   

                                              Vehicle Location Vehicle Type  \
Timestamp           Vehicle_Location                                          
2019-09-01 18:31:48 Space 3                            Space 3          Car   
                    Space 1                            Space 1          Car   
                    Space 2                            Space 2          Car   
                    NB right turn pocket  NB right turn pocket          Car   
                    SB bike lane                  SB bike lane          Car   

                                         Vehicle Characteristics  \
Timestamp           Vehicle_Location                               
2019-09-01 18:31:48 Space 3                                  NaN   
                    Space 1                                  NaN   
                    Space 2                                  NaN   
                    NB right turn pocket                     NaN   
                    SB bike lane                     Blinkers on   

                                                Transaction Characteristics  \
Timestamp           Vehicle_Location                                          
2019-09-01 18:31:48 Space 3                                             NaN   
                    Space 1                  Interaction with parking meter   
                    Space 2               No interaction with parking meter   
                    NB right turn pocket                                NaN   
                    SB bike lane                                        NaN   

                                         Bikeway Users Displaced  \
Timestamp           Vehicle_Location                               
2019-09-01 18:31:48 Space 3                                  NaN   
                    Space 1                                  NaN   
                    Space 2                                  NaN   
                    NB right turn pocket                     NaN   
                    SB bike lane                               6   

                                         LZ Space Avail Occupied while idle?  \
Timestamp           Vehicle_Location                                           
2019-09-01 18:31:48 Space 3                         NaN                False   
                    Space 1                         NaN                False   
                    Space 2                         NaN                False   
                    NB right turn pocket              0                False   
                    SB bike lane                      0                 True   

                                           CNS?   TNC?  \
Timestamp           Vehicle_Location                     
2019-09-01 18:31:48 Space 3               False  False   
                    Space 1               False  False   
                    Space 2               False  False   
                    NB right turn pocket   True  False   
                    SB bike lane          False  False   

                                                                                  Notes  \
Timestamp           Vehicle_Location                                                      
2019-09-01 18:31:48 Space 3                                                         NaN   
                    Space 1               looked at the meter but still decided to park   
                    Space 2                                                         NaN

In [14]:
alldays_timestamped.to_parquet('alldays_timestamped.parquet')